# Quantitative Value Strategy

In [3]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/home/adedapo/personal_project/daps05ayoade/disseration')
from algotrading.secrecy import API_TOKEN

In [4]:
def split_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [5]:
columns = ['Ticker', 'PERatio', 'PriceToBookRatio', 'PriceToSalesRatio', 'EVToEBITDA', 'EVToRevenue']
df_metrics = pd.DataFrame(columns = columns)
file_path = '/home/adedapo/personal_project/daps05ayoade/disseration/data/50_tech_stocks.csv'
symbol_df = pd.read_csv(file_path)
symbols = [ticker for ticker in symbol_df['Ticker']]

for symbol in symbols:
    try:
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_TOKEN}'
        data = requests.get(url).json()
        df_metrics = df_metrics.append(
                                        pd.Series([symbol,
                                                   data['PERatio'],
                                                   data['PriceToBookRatio'],
                                                   data['PriceToSalesRatioTTM'],
                                                   data['EVToEBITDA'],
                                                   data['EVToRevenue']],
                                                   index =columns),
                                                   ignore_index = True)
    except KeyError:
        print(f"KeyError for ticker: {symbol}")

In [6]:
df_metrics

,Ticker,PERatio,PriceToBookRatio,PriceToSalesRatio,EVToEBITDA,EVToRevenue
0,AAPL,32.02,44.63,5.51,17.53,5.92
1,MSFT,35.91,11.05,9.14,18.55,9.21
2,GOOG,25.91,4.864,4.179,12.16,4.057
3,AMZN,295.65,6.61,1.797,19.3,1.948
4,NVDA,220.84,19.59,14.79,56.0,14.59
5,TSLA,74.48,12.42,6.65,29.61,6.41
6,META,35.78,2.598,2.614,7.05,2.595
7,AVGO,27.5,9.71,6.81,12.93,7.46
8,ORCL,37.26,47.38,4.686,16.78,6.5
9,ADBE,47.14,10.99,8.89,22.68,9.13


In [7]:
from statistics import mean
from scipy import stats

# Define the metrics dictionary
metrics = {
    'PERatio': 'PEPercentile',
    'PriceToBookRatio': 'PBPercentile',
    'PriceToSalesRatio': 'PSPercentile',
    'EVToEBITDA': 'EV/EBITDAPercentile',
    'EVToRevenue': 'EV/RevenuePercentile'
}

# Calculating the percentiles for each metric
for row in df_metrics.index:
    for metric in metrics.keys():
        df_metrics.loc[row, metrics[metric]] = stats.percentileofscore(df_metrics[metric], df_metrics.loc[row, metric])/100

for row in df_metrics.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(df_metrics.loc[row, metrics[metric]])
    df_metrics.loc[row, 'QV Score'] = mean(value_percentiles)
qv_score = df_metrics['QV Score']

In [8]:
qv_score

0     0.504
1     0.580
2     0.376
3     0.332
4     0.344
5     0.604
6     0.460
7     0.588
8     0.522
9     0.608
10    0.336
11    0.544
12    0.656
13    0.512
14    0.432
15    0.384
16    0.504
17    0.820
18    0.380
19    0.512
20    0.196
21    0.436
22    0.484
23    0.568
24    0.444
25    0.360
26    0.550
27    0.738
28    0.296
29    0.364
30    0.376
31    0.656
32    0.592
33    0.400
34    0.900
35    0.436
36    0.544
37    0.512
38    0.628
39    0.796
40    0.636
41    0.650
42    0.784
43    0.468
44    0.784
45    0.460
46    0.360
47    0.320
48    0.436
49    0.328
Name: QV Score, dtype: float64